In [7]:
# -*- coding:utf-8 -*-
from theano import pp
from theano import tensor as T
from theano import function
from theano.printing import debugprint
x = T.dscalar('x')
y = x ** 2
gy = T.grad(y, x)
pp(gy)

'((fill((x ** TensorConstant{2}), TensorConstant{1.0}) * TensorConstant{2}) * (x ** (TensorConstant{2} - TensorConstant{1})))'

In [5]:
f = function([x], gy)
print f(4)

8.0


In [8]:
debugprint(f)

Elemwise{mul,no_inplace} [@A] ''   0
 |TensorConstant{2.0} [@B]
 |x [@C]


In [9]:
pp(f.maker.fgraph.outputs[0])

'(TensorConstant{2.0} * x)'

In [14]:
w = T.dscalar('w')
b = T.dscalar('b')
y = T.dscalar('y')
p = 1/(1+T.exp(- w * x - b))
logloss = y * T.log(p) - (1-y) * T.log(1-p)
logistic_grad = T.grad(logloss, x)
f = function([x,y,w,b], logistic_grad)
pp(f.maker.fgraph.outputs[0])

'Elemwise{Composite{(scalar_sigmoid((-Composite{((i0 * i1 * i2) - i3)}(i0, i1, i2, i3))) * ((i4 + (i5 * exp(Composite{((i0 * i1 * i2) - i3)}(i0, i1, i2, i3)))) - i5) * i1)}}(TensorConstant{-1.0}, w, x, b, TensorConstant{1.0}, y)'

In [16]:
z = T.grad(x, x)
pp(z)

'fill(x, TensorConstant{1.0})'

In [19]:
import theano
x = T.dvector('x')
y = x ** 2
J, updates = theano.scan(lambda i, y ,x : T.grad(y[i], x), sequences=T.arange(y.shape[0]),non_sequences=[y,x])
f = function([x], J, updates=updates)
print f([4,4])

[[ 8.  0.]
 [ 0.  8.]]


In [21]:
test, updates = theano.scan(lambda i, y ,x : y[i] ** x, sequences=T.arange(y.shape[0]),non_sequences=[y,x])
ftest = function([x,y], test, updates=updates)
print ftest([4,2],[1,2,3])

[[  1.   1.]
 [ 16.   4.]
 [ 81.   9.]]


In [24]:
jacob = theano.gradient.jacobian(y,x)
f = function([x], jacob)
print f([4,4])

[[ 8.  0.]
 [ 0.  8.]]


In [27]:
cost = y.sum()
hesse = theano.gradient.hessian(cost, x)
f = function([x], hesse)
print f([4,4])

[[ 2.  0.]
 [ 0.  2.]]


In [29]:
gy = T.grad(cost, x)
H, updates = theano.scan(lambda i, gy, x : T.grad(gy[i], x), sequences=T.arange(gy.shape[0]), non_sequences=[gy, x])
f = function([x], H, updates=updates)
print f([4,4])

[[ 2.  0.]
 [ 0.  2.]]


In [30]:
# Jacobian x vector
W = T.dmatrix('W')
V = T.dmatrix('V')
x = T.dvector('x')
y = T.dot(x, W)
JV = T.Rop(y, W, V) # R-operation (?) delta f(x)/delta x * v
f = function([W,V,x], JV)
print f([[1,1],[1,1]], [[2,2],[2,2]],[0,1])

[ 2.  2.]


In [32]:
jacob = theano.gradient.jacobian(y,x)
f = function([W, V, x], jacob * V)
print f([[1,1],[1,1]],[[2,2],[2,2]],[0,1])

[[ 2.  2.]
 [ 2.  2.]]


In [33]:
v = T.dvector('v')
VJ = T.Lop(y, W, v)
f = function([v,x], VJ)
print f([2,2],[0,1])

[[ 0.  0.]
 [ 2.  2.]]


In [34]:
# Hessian x vector
y = T.sum(x ** 2)
gy = T.grad(y, x)
vH = T.grad(T.sum(gy * v), x)
f = function([x,v], vH)
f([4,4],[2,2])

array([ 4.,  4.])

In [35]:
# because Hessian is symmetric, vH = (Hv.T).T
Hv = T.Rop(gy, x, v)
f = function([x,v], Hv)
f([4,4],[2,2])

array([ 4.,  4.])